In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ee

In [ ]:
# ee.Authenticate()
ee.Initialize()

In [ ]:
useful_crimes = gpd.read_file('filtered dataset/crimes_filtered.shp')
useful_crimes.to_crs('EPSG:4326', inplace=True)

In [ ]:
manz = gpd.read_file('nyct2020_23b/nyct2020.shp')
manz.to_crs('EPSG:4326', inplace=True)
manz.head(1)

In [ ]:
useful_crimes.head(1)

### 1. Importaciones y preprocesamiento

In [ ]:

poly = manz.envelope.geometry.iloc[0]
xy = list(poly.exterior.coords)
aoi = ee.Geometry.Polygon(xy)

In [ ]:
collection = ee.ImageCollection("LANDSAT/LC08/C01/T1").filterBounds(aoi).filterDate('2022-12-30', '2023-04-29')
images=collection.aggregate_array('system:id').getInfo()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
city.plot(ax=ax, color="darkgreen", zorder=0)

plt.title("Acceso Deficiente a Agua Potable Rural", size=20)
plt.xlabel("Easting (m)")
plt.ylabel("Northing (m)")
plt.show()


In [ ]:
useful_crimes.head()

In [ ]:
# alfonso :)